<html>
<body>
    <center> 
        <h1><u>Drosophila Species Categorizer</u></h1>
        <h3> Distinguishes D. mel from D. americanus and D. virilis</h3>
    </center>
</body>
</html>

In [1]:
### Import general libraries and pytorch

import os
import sys
from tqdm.notebook import tqdm
import json
import random
import pickle
import copy
from IPython.display import display
import ipywidgets as widgets
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import cv2

In [2]:
### Determine git path ####
git_dir = 'C:/Users/alisc/Documents/GitHub/Harvard_BAI'
print('Your github directory is :%s'%git_dir)

Your github directory is :C:/Users/alisc/Documents/GitHub/Harvard_BAI


In [3]:
project_folder = "%s/Final_Project"%git_dir

In [4]:
os.chdir(project_folder)

#### Load model loaders

In [5]:
### Making helper code under the folder res available. This includes loaders, models, etc. ###
sys.path.append('%s/res/'%git_dir)
from models.models import get_model
from loader.loader import get_loader

Models are being loaded from: C:\Users\alisc\Documents\GitHub\Harvard_BAI\res\models
Loaders are being loaded from: C:\Users\alisc\Documents\GitHub\Harvard_BAI\res\loader


### Specifying config for the model ###

In [6]:
wandb_config = {}
wandb_config['batch_size'] = 10
wandb_config['base_lr'] = 0.01
wandb_config['model_arch'] = 'ResNet152'
wandb_config['num_classes'] = 3
wandb_config['run_name'] = 'Final_Project'


wandb_config['use_gpu'] = 1
wandb_config['num_epochs'] = 100
wandb_config['git_dir'] = git_dir

### Data Loading and Creating Labels Dict ###

### Creates file lists for training (50%), validation (30%), and testing (20%) images

In [7]:
total_paths = {}
total_files = {}
iterator = 0
for path, directories, files in os.walk('%s/data/insect_species/'%project_folder): 
    if files != []:
        total_paths[iterator] = path
        total_files[iterator] = files
        iterator += 1

In [8]:
total_points = np.array(range(len(total_files)))
total_file_paths = {0:[], 1:[], 2:[]}
train_file_paths = {0:[], 1:[], 2:[]}
val_file_paths = {0:[], 1:[], 2:[]}
test_file_paths = {0:[], 1:[], 2:[]}

for species in range(len(total_files)):
    total_points[species] = len(total_files[species])
    for file in range(int(total_points[species]*0.5)):
        train_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
        total_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
    for file in range(int(total_points[species]*0.5)+1, int(total_points[species]*0.8)):
        val_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
        total_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
    for file in range(int(total_points[species]*0.8)+1, int(total_points[species])):
        test_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
        total_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])

In [9]:
labels_dictionary = {}

for species in range(len(total_file_paths)):
    for file_path in total_file_paths[species]:
        labels_dictionary[file_path] = species


Dumps the created labels_dictionary and training, val, and test data file paths for later reference.

In [10]:
with open('%s/data/labels_dictionary.p'%project_folder, 'wb') as F:
    pickle.dump(labels_dictionary, F)

with open('%s/data/train_file_list.txt'%project_folder, 'w') as filehandle:
    for i in range(3):
        for listitem in train_file_paths[i]:
            filehandle.write('%s\n' % listitem)

with open('%s/data/val_file_list.txt'%project_folder, 'w') as filehandle:
    for i in range(3):
        for listitem in val_file_paths[i]:
            filehandle.write('%s\n' % listitem)
            
with open('%s/data/test_file_list.txt'%project_folder, 'w') as filehandle:
    for i in range(3):
        for listitem in test_file_paths[i]:
            filehandle.write('%s\n' % listitem)

# Using a custom data-loader to store image paths, labels, and transforms. 

Use the 'cats_dogs_loader' for now, mainly a naming issue - the dataset formatting is absolutely fine.

In [11]:
file_list_loader = get_loader('cats_dogs_loader')

In [12]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
     'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [13]:
dsets = {}
dsets['train'] = file_list_loader('%s/data/train_file_list.txt'%project_folder, '%s/data/labels_dictionary.p'%project_folder, data_transforms['train'])
dsets['val'] = file_list_loader('%s/data/val_file_list.txt'%project_folder, '%s/data/labels_dictionary.p'%project_folder, data_transforms['val'])
dsets['test'] = file_list_loader('%s/data/test_file_list.txt'%project_folder, '%s/data/labels_dictionary.p'%project_folder, data_transforms['test'])

In [14]:
### Above, we created datasets. Now, we will pass them into pytorch's inbuild dataloaders, 
### these will help us load batches of data for training.
dset_loaders = {}
dset_loaders['train'] = torch.utils.data.DataLoader(dsets['train'], batch_size=wandb_config['batch_size'], shuffle = True, num_workers=2,drop_last=False)
dset_loaders['val'] = torch.utils.data.DataLoader(dsets['val'], batch_size=wandb_config['batch_size'], shuffle = False, num_workers=2,drop_last=False)
dset_loaders['test'] = torch.utils.data.DataLoader(dsets['test'], batch_size=wandb_config['batch_size'], shuffle = True, num_workers=2,drop_last=False)

In [15]:
data_sizes = {}
data_sizes['train'] = len(dsets['train'])
data_sizes['val'] = len(dsets['val'])
data_sizes['test'] = len(dsets['test'])

# Load a ResNet 152 model

In [16]:
### Loads a ResNet model with 152 layers as determined in loaders.py

model = get_model('ResNet152', 1000)
in_filters = model.fc.in_features
model.fc = nn.Linear(in_features=in_filters, out_features=wandb_config['num_classes'])
model.cuda();

#### Train, test, and return the best model weights.

In [17]:
def model_pipeline(model, criterion, optimizer, dset_loaders, dset_sizes, hyperparameters):
    config = wandb_config
    best_model = model
    best_acc = 0.0

    print(config)

    for epoch_num in tqdm(range(config['num_epochs'])):
        model = train_model(model, criterion, optimizer, dset_loaders, dset_sizes, config)
        best_acc, best_model = val_model(model, best_acc, best_model, dset_loaders, dset_sizes, config)

    return best_model

In [18]:
def train_model(model, criterion, optimizer, dset_loaders, dset_sizes, configs):
    best_model = model
    best_acc = 0.0

    
    ### This tells python to track gradients. While testing weights aren't updated hence they are not stored.
    model.train() 
    running_loss = 0.0
    running_corrects = 0
    iters = 0
    
    
    ### We loop over the data loader we created above. Simply using a for loop.
    for data in enumerate(dset_loaders['train']):
        inputs, labels = data[1][0], data[1][1]
        
        ### If you are using a gpu, then script will move the loaded data to the GPU. 
        ### If you are not using a gpu, ensure that wandb_configs['use_gpu'] is set to False above.
        if wandb_config['use_gpu']:
            inputs = inputs.float().cuda()
            labels = labels.long().cuda()
        else:
            print('WARNING: NOT USING GPU!')
            inputs = inputs.float()
            labels = labels.long()

        
        ### We set the gradients to zero, then calculate the outputs, and the loss function. 
        ### Gradients for this process are automatically calculated by PyTorch.
        
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        loss = criterion(outputs, labels)
        
        
        ### At this point, the program has calculated gradient of loss w.r.t. weights of our NN model.
        loss.backward()
        optimizer.step()
        
        ### optimizer.step() updated the models weights using calculated gradients.
        
        
        iters += 1
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = float(running_loss) / dset_sizes['train']
    epoch_acc = float(running_corrects) / float(dset_sizes['train'])
    return model



In [19]:
def val_model(model, best_acc, best_model, dset_loaders, dset_sizes, configs):
    model.eval() ### tells pytorch to not store gradients as we won't be updating weights while testing.

    running_corrects = 0
    iters = 0   
    for data in enumerate(dset_loaders['val']):
        inputs, labels = data[1][0], data[1][1]
        
        if wandb_config['use_gpu']:
            inputs = inputs.float().cuda()
            labels = labels.long().cuda()
        else:
            print('WARNING: NOT USING GPU!')
            inputs = inputs.float()
            labels = labels.long()

        
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        
        iters += 1
        running_corrects += torch.sum(preds == labels.data)


    epoch_acc = float(running_corrects) / float(dset_sizes['val'])
   
    ### Code is very similar to train set. One major difference, we don't update weights. 
    ### We only check the performance is best so far, if so, we save this model as the best model so far.
    
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model = copy.deepcopy(model)
    
    return best_acc, best_model
    

In [20]:
### Set model save path
save_path = '%s/saved_models/%s_final.pt'%(wandb_config['git_dir'], wandb_config['run_name'])

In [ ]:
### Criterion is simply specifying what loss to use. Here we choose cross entropy loss. 
criterion = nn.CrossEntropyLoss()

### tells what optimizer to use. There are many options, we here choose Adam.
### the main difference between optimizers is that they vary in how weights are updated based on calculated gradients.
optimizer_ft = optim.Adam(model.parameters(), lr = wandb_config['base_lr'])

if wandb_config['use_gpu']:
    criterion.cuda()
    model.cuda()
    

### Creating the folder where our models will be saved.
if not os.path.isdir("%s/saved_models/"%wandb_config['git_dir']):
    os.mkdir("%s/saved_models/"%wandb_config['git_dir'])
    
### Let's run it all, and save the final best model.
best_final_model = model_pipeline(model, criterion, optimizer_ft, dset_loaders, data_sizes, wandb_config)

with open(save_path,'wb') as F:
    torch.save(best_final_model,F)
print('Best model saved in %s'%save_path)

{'batch_size': 10, 'base_lr': 0.01, 'model_arch': 'ResNet152', 'num_classes': 3, 'run_name': 'Final_Project', 'use_gpu': 1, 'num_epochs': 100, 'git_dir': 'C:/Users/alisc/Documents/GitHub/Harvard_BAI'}


Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing ep

## Reload saved model and determine classification accuracy

In [21]:
# Load model as model_normal to later compare with model_augmented

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_normal = torch.load(save_path)
model_normal.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Get model predictions for test set

In [79]:
running_corrects = 0
iters = 0   
for data in enumerate(dset_loaders['test']):
    inputs, labels = data[1][0], data[1][1]

    inputs = inputs.float().cuda()
    labels = labels.long().cuda()

    outputs = model_normal(inputs)
    _, preds = torch.max(outputs.data, 1)

    iters += 1
    running_corrects += torch.sum(preds == labels.data)

model_norm_acc = float(running_corrects) / float(data_sizes['test'])
print('Model_norm prediction accuracy is %s'%model_norm_acc)

Model_norm prediction accuracy is 0.17427385892116182


# Repeat above for model with augmented data

### Augment training, validation, and test set images

In [96]:
image_path_augmented = project_folder + '/data/Images_Drosophila_Augmented/'
if not os.path.isdir(image_path_augmented+'0'):
    os.mkdir(image_path_augmented+'0')
if not os.path.isdir(image_path_augmented+'1'):
    os.mkdir(image_path_augmented+'1')
if not os.path.isdir(image_path_augmented+'2'):
    os.mkdir(image_path_augmented+'2')

filename = 0
for species in range(len(total_file_paths)):
    for image in range(len(total_file_paths[species])):
        filename += 1
        image_original = cv2.imread(total_file_paths[species][image])
        image_rotated_90 = cv2.rotate(image_original, cv2.ROTATE_90_CLOCKWISE)
        image_rotated_180 = cv2.rotate(image_original, cv2.ROTATE_180)
        
        hsv = cv2.cvtColor(image_original, cv2.COLOR_BGR2HSV)
        hsv = np.array(hsv, dtype = np.float64)
        hsv[:,:,1] = hsv[:,:,1]*2
        hsv[:,:,1][hsv[:,:,1]>255]  = 255
        hsv[:,:,2] = hsv[:,:,2]*2
        hsv[:,:,2][hsv[:,:,2]>255]  = 255
        hsv = np.array(hsv, dtype = np.uint8)
        image_bright = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        image_flippedh = cv2.flip(image_original, 1)
        image_flippedv = cv2.flip(image_original, 0)
        
        cv2.imwrite( (image_path_augmented+str(species)+'/'+str(filename)+'.jpg'), image_original)
        cv2.imwrite( (image_path_augmented+str(species)+'/'+str(filename)+'_90.jpg'), image_rotated_90)
        cv2.imwrite( (image_path_augmented+str(species)+'/'+str(filename)+'_180.jpg'), image_rotated_180)
        cv2.imwrite( (image_path_augmented+str(species)+'/'+str(filename)+'_bright.jpg'), image_bright)
        cv2.imwrite( (image_path_augmented+str(species)+'/'+str(filename)+'_flippedh.jpg'), image_flippedh)
        cv2.imwrite( (image_path_augmented+str(species)+'/'+str(filename)+'_flippedv.jpg'), image_flippedv)

## Set up augmented model identical to normal model

In [97]:
total_paths = {}
total_files = {}
iterator = 0
for path, directories, files in os.walk(image_path_augmented): 
    if files != []:
        total_paths[iterator] = path
        total_files[iterator] = files
        iterator += 1
total_points = np.array(range(len(total_files)))
total_file_paths = {0:[], 1:[], 2:[]}
train_file_paths = {0:[], 1:[], 2:[]}
val_file_paths = {0:[], 1:[], 2:[]}
test_file_paths = {0:[], 1:[], 2:[]}

for species in range(len(total_files)):
    total_points[species] = len(total_files[species])
    for file in range(int(total_points[species]*0.5)):
        train_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
        total_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
    for file in range(int(total_points[species]*0.5)+1, int(total_points[species]*0.8)):
        val_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
        total_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
    for file in range(int(total_points[species]*0.8)+1, int(total_points[species])):
        test_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])
        total_file_paths[species].append(total_paths[species] + '/' + list(total_files[species])[file])

labels_dictionary = {}

for species in range(len(total_file_paths)):
    for file_path in total_file_paths[species]:
        labels_dictionary[file_path] = species
with open('%s/data/labels_dictionary_augmented.p'%project_folder, 'wb') as F:
    pickle.dump(labels_dictionary, F)

with open('%s/data/train_file_list_augmented.txt'%project_folder, 'w') as filehandle:
    for i in range(3):
        for listitem in train_file_paths[i]:
            filehandle.write('%s\n' % listitem)

with open('%s/data/val_file_list_augmented.txt'%project_folder, 'w') as filehandle:
    for i in range(3):
        for listitem in val_file_paths[i]:
            filehandle.write('%s\n' % listitem)
            
with open('%s/data/test_file_list_augmented.txt'%project_folder, 'w') as filehandle:
    for i in range(3):
        for listitem in test_file_paths[i]:
            filehandle.write('%s\n' % listitem)
            
file_list_loader = get_loader('cats_dogs_loader')

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
     'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dsets = {}
dsets['train'] = file_list_loader('%s/data/train_file_list_augmented.txt'%project_folder, '%s/data/labels_dictionary_augmented.p'%project_folder, data_transforms['train'])
dsets['val'] = file_list_loader('%s/data/val_file_list_augmented.txt'%project_folder, '%s/data/labels_dictionary_augmented.p'%project_folder, data_transforms['val'])
dsets['test'] = file_list_loader('%s/data/test_file_list_augmented.txt'%project_folder, '%s/data/labels_dictionary_augmented.p'%project_folder, data_transforms['test'])

dset_loaders = {}
dset_loaders['train'] = torch.utils.data.DataLoader(dsets['train'], batch_size=wandb_config['batch_size'], shuffle = True, num_workers=2,drop_last=False)
dset_loaders['val'] = torch.utils.data.DataLoader(dsets['val'], batch_size=wandb_config['batch_size'], shuffle = False, num_workers=2,drop_last=False)
dset_loaders['test'] = torch.utils.data.DataLoader(dsets['test'], batch_size=wandb_config['batch_size'], shuffle = True, num_workers=2,drop_last=False)

data_sizes = {}
data_sizes['train'] = len(dsets['train'])
data_sizes['val'] = len(dsets['val'])
data_sizes['test'] = len(dsets['test'])

### Loads a ResNet model with 152 layers as determined in loaders.py
model_augmented = get_model('ResNet152', 1000)
in_filters = model_augmented.fc.in_features
model_augmented.fc = nn.Linear(in_features=in_filters, out_features=wandb_config['num_classes'])
model_augmented.cuda();

# pipeline functions are already defined above

### Set model save path
save_path = '%s/saved_models/%s_augmented.pt'%(wandb_config['git_dir'], wandb_config['run_name'])

### Train augmented model

In [98]:
# Run augmented model
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model.parameters(), lr = wandb_config['base_lr'])

if wandb_config['use_gpu']:
    criterion.cuda()
    model.cuda()
    
if not os.path.isdir("%s/saved_models/"%wandb_config['git_dir']):
    os.mkdir("%s/saved_models/"%wandb_config['git_dir'])
    
### Let's run it all, and save the final best model.
best_final_model_augmented = model_pipeline(model_augmented, criterion, optimizer_ft, dset_loaders, data_sizes, wandb_config)

with open(save_path,'wb') as F:
    torch.save(best_final_model_augmented,F)
print('Best model saved in %s'%save_path)


{'batch_size': 10, 'base_lr': 0.01, 'model_arch': 'ResNet152', 'num_classes': 3, 'run_name': 'Final_Project', 'use_gpu': 1, 'num_epochs': 100, 'git_dir': 'C:/Users/alisc/Documents/GitHub/Harvard_BAI'}


Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing epoch...
Starting training epoch...
Starting testing ep

In [100]:
# Load augmented model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_augmented = torch.load(save_path)
model_normal.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Get augmented model predictions for test set

In [101]:
running_corrects = 0
iters = 0   
for data in enumerate(dset_loaders['test']):
    inputs, labels = data[1][0], data[1][1]

    inputs = inputs.float().cuda()
    labels = labels.long().cuda()

    outputs = model_augmented(inputs)
    _, preds = torch.max(outputs.data, 1)

    iters += 1
    running_corrects += torch.sum(preds == labels.data)

model_augmented_acc = float(running_corrects) / float(data_sizes['test'])
print('Model_norm prediction accuracy is %s'%model_augmented_acc)

Model_norm prediction accuracy is 0.1743498817966903
